# Age of Empires 2 Player Model

This model is used to rank invidividual players in team games. This model will allow us to better balance teams by calculating the probability that team wins before we actually play.

Disclaimer: I am not a data scientist, who fully undertands the underlying math.

## Todo:
- Create test cases
- Load data from Google Sheet instead of local CSV
- Determine what EDA should be done
- Fix GridSearchCV to LogisticRegression import
- Explore adding a time component to factor in player improvement
- Determine how to better input data for predicting
- Build other classifiers

## Import dependencies

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
import matplotlib.pyplot as plt
import numpy as np

## Import Data

In [2]:
df = pd.read_csv("sample_data.csv")

# Designate all columns that are not `Outcome` as features and `Outcome` as target
X = df.loc[:, df.columns != 'Outcome']
y = df.Outcome

df.head(2)

,Shaq,Gray,Rushi,Marc,Peter,Pat,Sam,Ori,Vic,Ardy,Chad,Pat_Jr,Pat_Jr_Jr,Matt_M,Ben,Mikey,Evan,Medium_AI,Extra_Team,Outcome
0,1,0,-1,-1,1,0,-1,0,0,0,0,0,0,0,0,0,0,0,-1,-1
1,1,0,-1,0,-1,1,-1,1,0,0,0,0,0,0,0,0,0,0,0,-1


## Explore Data
This is where I should explore data. I haven't done any EDA since I created this dataset.

## Split data
Normally, I would split the data into a training set and validation set. The validation set is for checking the accuracy of the best tuned model that results from cross-validation. HOWEVER, we are working with a really small dataset. Rather than hold out datafor validation, we will assess the performance of the model through the out of sample cross validation results

In [3]:
#X_train,X_validate,y_train,y_validate=train_test_split(X,y,test_size=0.33,random_state=0)

## Double data
Since assigning teams is random, we want to ensure that the dataset is balanced. For example, when I record data, I generally always put myself as the home team (code as `1`). We mitigate this by not having an intercept term in our model. To be safe, we will still double the dataset by inverting all the records and concatenating to the orginal dataset.

Doubling happens after splitting. Therefore we would need to double the training and validation sets. We use helper functions for readability.

In [4]:
def invert_dataframe(original_dataframe):
    """Inverts the dataframe by simply multiplying all values by -1.

    Args:
        original_datatframe (df): The dataframe to be inverted.

    Returns:
        inverted_dataframe (df): The inverted dataframe.

    """
    inverted_dataframe = original_dataframe.multiply(-1)
    return inverted_dataframe

def combine_dataframe(first_dataframe, second_dataframe):
    """Combines the dataframes. Assumes that both dataframes have the same columns

    Args:
        first_datatframe (df): The first dataframe to be combined.
        second_datatframe (df): The second dataframe to be combined.

    Returns:
        combined_dataframe (df): The combined dataframe.

    """
    combined_dataframe = pd.concat([first_dataframe, second_dataframe])
    return combined_dataframe

def invert_and_combine(original_dataframe):
    """Inverts and combines the dataframes. Assumes that both dataframes have the same columns

    Args:
        original_dataframe (df): The dataframe to be inverted and combined with the original.

    Returns:
        new_dataframe (df): The combined dataframe.

    """
    inverted_dataframe = invert_dataframe(original_dataframe)
    new_dataframe = combine_dataframe(original_dataframe, inverted_dataframe)
    return new_dataframe
    

In [5]:
X = invert_and_combine(X)
y = invert_and_combine(y)

# These are commented out because we are not using a validation set
# X_train = invert_and_combine(X_train)
# X_validate = invert_and_combine(X_validate)
# y_train = invert_and_combine(y_train)
# y_validate = invert_and_combine(y_validate)

## Cross Validation
We will use 3 folds cross validation and GridSearch to determine the optimal hyper parameters for the logistic regression. The parameters we will search for is C (regularization and expressed as 1/lambda). We will assume `l2` penalty (default), `liblinear` solver (default), and no fit_intercept.

In [6]:
C = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]
penalty = ['l2']
solver = ['liblinear']
fit_intercept = [False]
param_grid = dict(C=C, penalty=penalty, fit_intercept=fit_intercept, solver=solver)

We will now execute the GridSearch over three folds. We will use `accuracy` to assess the performance of the hyperparameters.

In [7]:
lr = LogisticRegression()
grid = GridSearchCV(estimator=lr, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
best_model_from_cv = grid.fit(X, y)

### Inspect cross validation
Here we inspect the accuracy and standard deviation for each run. Found code [here](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html)

In [8]:
means = best_model_from_cv.cv_results_['mean_test_score']
stds = best_model_from_cv.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, best_model_from_cv.cv_results_['params']):
    print("{0:.3f} (+/-{1:.3f}) for {2}".format(mean, std * 2, params))
    
print("Best model according to grid search: {0} using {1}".format(round(best_model_from_cv.best_score_,2), best_model_from_cv.best_params_))

0.539 (+/-0.070) for {'C': 0.001, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}
0.559 (+/-0.058) for {'C': 0.01, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}
0.624 (+/-0.187) for {'C': 0.1, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}
0.773 (+/-0.108) for {'C': 1.0, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}
0.838 (+/-0.077) for {'C': 10.0, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}
0.838 (+/-0.099) for {'C': 100.0, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}
0.838 (+/-0.099) for {'C': 1000.0, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}
0.838 (+/-0.099) for {'C': 10000.0, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}
Best model according to grid search: 0.84 using {'C': 100.0, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}


### Picking the right model
There is a balance between model generalization and accuracy. Too low penalization (i.e. a high C) means that the model could be overfitting. However, low penalization means that the model has higher accuracy. Therefore, we will pick the model that has the lowest C, while still within the the bounds of the highest accuracy model. Specifically, we want the lowest C that is one standard error within the best C's performance.

In [9]:
def find_target_accuracy(best_model_from_cv):
    """Finds the target accuracy that the second best model has to exceed

    Args:
        best_model_from_cv (GridSearchCV): The object that has the info from cross validation.

    Returns:
        target_accuracy (float): The best model's accuracy descreased by its standard deviation

    """
    best_cv_model_index = best_model_from_cv.best_index_
    best_cv_model_mean_accuracy = best_model_from_cv.cv_results_['mean_test_score'][best_cv_model_index]
    best_cv_model_std_accuracy = best_model_from_cv.cv_results_['std_test_score'][best_cv_model_index]
    target_accuracy = best_cv_model_mean_accuracy - best_cv_model_std_accuracy
    return target_accuracy


def find_final_model_params(best_model_from_cv):
    """Finds the parameters for the final model that will be trained on all data. 
    We want to see whether there is a model that has more generalization but satisfactory accuracy

    Args:
        best_model_from_cv (GridSearchCV): The object that has the info from cross validation.

    Returns:
        best_model_parameters (dict): The best model's parameters

    """
    target_accuracy = find_target_accuracy(best_model_from_cv)
    index_of_final_model = loop_through_cv_to_find_index_of_final_model(best_model_from_cv, target_accuracy)
    best_model_parameters = best_model_from_cv.cv_results_['params'][index_of_final_model]
    return best_model_parameters


def loop_through_cv_to_find_index_of_final_model(best_model_from_cv, target_accuracy):
    """We want to see whether there is a model that has more generalization but satisfactory accuracy.
    We loop through the results until we find a model that has a satisfactory accuracy.
    The loop will stop at the best model in case none of the models with higher generalization are
    satisfactory. This function is assuming that the models are sorted.

    Args:
        best_model_from_cv (GridSearchCV): The object that has the info from cross validation.
        target_accuracy (float): The cv best model's accuracy descreased by its standard deviation

    Returns:
        target_index (int): The index of the final model

    """
    target_index = best_model_from_cv.best_index_
    for i, score in enumerate(best_model_from_cv.cv_results_['mean_test_score']):
        if(score > target_accuracy and i < target_index):
            target_index = i
            print("Found better model: {0:.3f} (+/-{1:.3f}) using {2}".format(best_model_from_cv.cv_results_['mean_test_score'][target_index],
                                                                      2*best_model_from_cv.cv_results_['std_test_score'][target_index],
                                                                      best_model_from_cv.cv_results_['params'][target_index]))
            break
    return target_index
    

In [10]:
best_model_parameters = find_final_model_params(best_model_from_cv)

Found better model: 0.838 (+/-0.077) using {'C': 10.0, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'liblinear'}


# Assess performance
Normally, we would now use the tuned hyperparameters to ensure accuracy on the validation set. As a reminder, the model is trained on the training set and the scores are computed on the validation set. HOWEVER, as mentioned, we would rather use all of our limited data for model building, so we will not assess performance on a validation set.

In [11]:
# final_model_with_only_training_data = = LogisticRegression(penalty=best_model_parameters['penalty'], 
#                                  C=best_model_parameters['C'],
#                                  fit_intercept=best_model_parameters['fit_intercept'],
#                                  solver=best_model_parameters['solver'])

# final_model_with_only_training_data = final_model_with_only_training_data.fit(X, y)
# y_true, y_pred = y_validate, final_model_with_only_training_data.predict(X_validate)
# print(classification_report(y_true, y_pred))

We can also use a ROC curve to visualize performance. The more above the diagonal, the better. More info [here](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5)

In [12]:
# logit_roc_auc = roc_auc_score(y_validate, final_model_with_only_training_data.predict(X_validate))
# fpr, tpr, thresholds = roc_curve(y_validate, final_model_with_only_training_data.predict_proba(X_validate)[:,1])
# plt.figure()
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic')
# plt.legend(loc="lower right")
# plt.show()

## Create final model
Since we are happy with the performance of our model on the validation set, we will re-fit it with all the data. There is no concern of overfitting because we already validated against data the model hadn't seen. Since we removed the validation set, the final model is the same as the model that resulted from the hyperparameter tuning.

In [13]:
# TODO: must be a cleaner way to import GridSearchCV into LogisticRegression
final_model_with_all_data = LogisticRegression(penalty=best_model_parameters['penalty'], 
                                 C=best_model_parameters['C'],
                                 fit_intercept=best_model_parameters['fit_intercept'],
                                 solver=best_model_parameters['solver'])

final_model_with_all_data = final_model_with_all_data.fit(X, y)

We will output the final coefficients to see how players are ranked and with what magnitude. We need to do some busy work to output a list of coefficients.

In [14]:
features = list(df.columns)
features.remove("Outcome")
[coef] = final_model_with_all_data.coef_.tolist()

rounded_coef = []
for number in coef:
    rounded_number = round(number, 2)
    rounded_coef.append(rounded_number)

x = zip(rounded_coef, features)
print("Final model coefficients are: {0}".format(sorted(list(x))))

Final model coefficients are: [(-2.8, 'Mikey'), (-2.67, 'Chad'), (-2.53, 'Sam'), (-1.92, 'Ori'), (-1.81, 'Matt_M'), (-1.67, 'Marc'), (-1.23, 'Evan'), (-0.3, 'Ben'), (-0.02, 'Pat_Jr_Jr'), (0.14, 'Medium_AI'), (0.98, 'Pat'), (1.13, 'Pat_Jr'), (1.35, 'Peter'), (1.44, 'Shaq'), (1.52, 'Ardy'), (2.74, 'Gray'), (2.92, 'Extra_Team'), (3.79, 'Vic'), (4.61, 'Rushi')]


## Predictions
Ultimately, we want to use this model to determine the probability of a game. Each value in the array corresponds to a person. For example, the first number is Shaq, the second number is Gray, etc.

Here we have modeled the probability that Marc (-1) beats Rushi (1).

In [15]:
result = final_model_with_all_data.predict_proba([[0, 0, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
print("The probability that Marc beats Rushi is {0}%".format(round(result[0][0]*100,2)))

The probability that Marc beats Rushi is 0.19%


Here we have modeled the probability that Shaq (-1) beats Gray (1).

In [16]:
result = final_model_with_all_data.predict_proba([[-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
print("The probability that Shaq beats Gray is {0}%".format(round(result[0][0]*100,2)))

The probability that Shaq beats Gray is 21.43%


Here we have modeled the probaility that Shaq (-1) and Gray (-1) beat Rushi

In [17]:
result = final_model_with_all_data.predict_proba([[-1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]])
print("The probability that Shaq and Gray beat Rushi is {0}%".format(round(result[0][0]*100,2)))

The probability that Shaq and Gray beat Rushi is 92.4%


Here we have modeled the probaility that Marc (-1) beats Sam (1)

In [18]:
result = final_model_with_all_data.predict_proba([[0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
print("The probability that Marc beats Sam is {0}%".format(round(result[0][0]*100,2)))

The probability that Marc beats Sam is 70.31%


Here we have modeled the probaility that Marc (-1) and Sam (-1) beat Rushi (1)

In [19]:
result = final_model_with_all_data.predict_proba([[0, 0, 1, -1, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]])
print("The probability that Marc and Sam beat Rushi is {0}%".format(round(result[0][0]*100,2)))

The probability that Marc and Sam beat Rushi is 0.27%


Here we have modeled the probaility that Vic (-1) beats Rushi (1)

In [20]:
result = final_model_with_all_data.predict_proba([[0, 0, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
print("The probability that Vic beats Rushi is {0}%".format(round(result[0][0]*100,2)))

The probability that Vic beats Rushi is 30.72%
